# PRÁCTICA GUIADA 2: Clasificadores Ada Boost y Gradient Boosting


## 1. Introducción

El objetivo de esta práctica es comenzar a aproximarnos al uso y a la evaluación de los métodos algoritmos de Boosting.

Recordemos: 

* La idea fundamental de los algoritmos basados en la noción de boosting era tratar de combinar muchos clasificadores débiles en un clasificador fuerte. 

* La principal diferencia con los métodos de Bagging era la forma en que se construye el ensamble: Boosting trata de mejorar la performance concentrándose de alguna forma en los casos con mayor error de entrenamiento.

### `AdaBoostClassifier()`

La idea central de AdaBoost es construir un ensamble de week learners y, en cada iteración ir incrementando el peso de los casos mal clasificados. La implementación den Scikit-Learn toma los siguientes parámetros:

* `base_estimator`: análogo al caso de `BaggingClassifier()`, el estimador sobre el cual se va a construir el ensamble. Por efecto, son árboles de decisión.
* `n_estimators`: el máximo de iteraciones
* `learning_rate`: el peso que va a tener la predicción de cada árbol en el ensamble final


### `GradientBoostingClassifier()`

Se trata de una generalización del algoritmo general de Boosting para cualquier tipo de función de pérdida diferenciable. En cada etapa, se fitea un árbol de decisión pero se realiza sobre los residuos del árbol anterior. Es decir, se busca corregir las estimaciones entrenando nuevos clasificadores sobre los "residuos" (la diferencia entre el valor observado y el valor predico ($y - \hat{y}$)

Los argumentos que toma como input son ya conocidos:

* `learning_rate`: el peso que va a tener la predicción de cada árbol en el ensamble final

* `n_estimators`: el máximo de iteraciones
* `criterion`: define el criterio de impureza para evaluar la calidad de las particiones
* `max_features`: la cantidad de features que extraerá para entrenar cada `base_estimator`. Por default es igual a `sqrt(X.shape[1])`
* `bootstrap` y `bootstrap_features`: controla si tanto los n_samples como las features son extraidos con reposición.
* `max_depth`: la pronfundidad máxima del árbol
* `min_samples_leaf`: el número mínimo de n_samples para constituir una hoja del árbol (nodo terminal)
* `min_samples_split`: el número mínimo de n_samples para realizar un split.

Utilizaremos como punto de partida el mismo código que utilizamos anteriormente:

In [1]:
import pandas as pd

df = pd.read_csv('../Data/car.csv')

In [2]:
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(df['acceptability'])
X = pd.get_dummies(df.drop('acceptability', axis=1), drop_first=True)

In [3]:
from sklearn.model_selection import cross_val_score,StratifiedKFold
cv = StratifiedKFold(n_splits=3, random_state=41, shuffle=True)

Ahora vamos a inicializar el clasificador de árbol de decisión y evaluar su rendimiento:

In [4]:
from sklearn.tree import DecisionTreeClassifier

def evaluar_rendimiento(modelo, nombre):
    s = cross_val_score(modelo, X, y, cv=cv, n_jobs=-1)
    print("Rendimiento de {}:\t{:0.3} ± {:0.3}".format( \
        nombre, s.mean().round(3), s.std().round(3)))
    
    
dt = DecisionTreeClassifier(class_weight='balanced')

evaluar_rendimiento(dt,"Árbol de decisión")

Rendimiento de Árbol de decisión:	0.898 ± 0.01


Ahora ustedes evalúen los siguientes clasificadores:
 * AdaBoostClassifier
 * GradientBoostingClassifier

In [5]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
ab = AdaBoostClassifier()
gb = GradientBoostingClassifier()
evaluar_rendimiento(ab, "AdaBoostClassifier")
evaluar_rendimiento(gb, "GradientBoostingClassifier")

Rendimiento de AdaBoostClassifier:	0.845 ± 0.01
Rendimiento de GradientBoostingClassifier:	0.955 ± 0.011


Puede verse, entonces, que AdaBoost performa bastante peor (al menos utilizando los parámetros por defecto). De esta forma, podríamos tratar de tunear los hiperparámetros para hacerlo funcionar mejor. Haremos esto en el LAB con otro dataset.